In [53]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
csv = pd.read_csv("healthcare-dataset-stroke-data\healthcare-dataset-stroke-data.csv",na_values="Unknown",encoding='utf-8')
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     3566 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [54]:
import numpy as np
from sklearn.linear_model import LinearRegression
#exclude bmi from training data and remove ids
y_column = csv.loc[csv['bmi'].notna(), "bmi"]
x_columns = csv[csv['bmi'].notna()]
x_columns.drop(axis=1,labels='bmi',inplace=True)
x_columns.drop(axis=1,labels='id',inplace=True)
csv = csv.drop(labels='id',axis=1)
print(y_column.head())
x_columns.head()

0    36.6
2    32.5
3    34.4
4    24.0
5    29.0
Name: bmi, dtype: float64
E:\Python\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,formerly smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,never smoked,1
5,Male,81.0,0,0,Yes,Private,Urban,186.21,formerly smoked,1


In [55]:
from sklearn import preprocessing
import numpy as np
#need to preprocess labels into numbers
le = preprocessing.LabelEncoder()
x_columns.loc[:,'gender'] = le.fit_transform(x_columns["gender"])
x_columns.loc[:,'ever_married'] = le.fit_transform(x_columns["ever_married"])
x_columns.loc[:,'work_type'] = le.fit_transform(x_columns["work_type"])
x_columns.loc[:,'Residence_type'] = le.fit_transform(x_columns["Residence_type"])
x_columns.loc[:,'smoking_status'] = le.fit_transform(x_columns["smoking_status"])
x_columns.info()

csv.loc[:,'gender'] = le.fit_transform(csv["gender"])
csv.loc[:,'ever_married'] = le.fit_transform(csv["ever_married"])
csv.loc[:,'work_type'] = le.fit_transform(csv["work_type"])
csv.loc[:,'Residence_type'] = le.fit_transform(csv["Residence_type"])
csv.loc[:,'smoking_status'] = le.fit_transform(csv["smoking_status"])
csv.drop(labels='bmi',axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 5109
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4909 non-null   int32  
 1   age                4909 non-null   float64
 2   hypertension       4909 non-null   int64  
 3   heart_disease      4909 non-null   int64  
 4   ever_married       4909 non-null   int32  
 5   work_type          4909 non-null   int32  
 6   Residence_type     4909 non-null   int32  
 7   avg_glucose_level  4909 non-null   float64
 8   smoking_status     4909 non-null   int32  
 9   stroke             4909 non-null   int64  
dtypes: float64(2), int32(5), int64(3)
memory usage: 326.0 KB


In [56]:
model = LinearRegression().fit(x_columns,y_column)
predictedBmi = model.predict(csv)
csv.fillna(value={'bmi':pd.Series(predictedBmi)},inplace=True)
model.score(x_columns,y_column)

0.1914729136040998

In [57]:
# Use numpy to convert to arrays
import numpy as np

# Labels are the values we want to predict
labels = np.array(csv['stroke'])
# Remove the labels from the csv
# axis 1 refers to the columns
csv= csv.drop('stroke', axis = 1)
# Saving feature names for later use
feature_list = list(csv.columns)
csv.fillna(value={'bmi':pd.Series(predictedBmi)},inplace=True)
# Convert to numpy array
csv = np.array(csv)

In [58]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(csv, labels, test_size = 0.25, random_state = 42)


In [59]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [60]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

In [61]:
predictions = predictions.round()
from sklearn.metrics import f1_score
print(f1_score(test_labels, predictions,average=None))
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(test_labels, predictions)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

[0.96593674 0.06666667]
Average precision-recall score: 0.07
